In [70]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [71]:
!pip install datasets nltk pandas

import random
import pandas as pd
from datasets import load_dataset
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import sqlite3
import re
from tqdm import tqdm
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [72]:
import pandas as pd
from datasets import load_dataset
from concurrent.futures import ThreadPoolExecutor, as_completed
import itertools

# Choose a subset of configs
configs = [
    'raw_review_All_Beauty', 'raw_review_Toys_and_Games', 'raw_review_Cell_Phones_and_Accessories',
    'raw_review_Industrial_and_Scientific', 'raw_review_Gift_Cards', 'raw_review_Musical_Instruments',
    'raw_review_Electronics', 'raw_review_Handmade_Products', 'raw_review_Arts_Crafts_and_Sewing',
    'raw_review_Baby_Products', 'raw_review_Health_and_Household', 'raw_review_Office_Products',
    'raw_review_Digital_Music', 'raw_review_Grocery_and_Gourmet_Food', 'raw_review_Sports_and_Outdoors',
    'raw_review_Home_and_Kitchen', 'raw_review_Subscription_Boxes', 'raw_review_Tools_and_Home_Improvement',
    'raw_review_Pet_Supplies', 'raw_review_Video_Games', 'raw_review_Kindle_Store', 'raw_review_Clothing_Shoes_and_Jewelry',
    'raw_review_Patio_Lawn_and_Garden', 'raw_review_Unknown', 'raw_review_Books', 'raw_review_Automotive',
    'raw_review_CDs_and_Vinyl', 'raw_review_Beauty_and_Personal_Care', 'raw_review_Amazon_Fashion',
    'raw_review_Magazine_Subscriptions', 'raw_review_Software', 'raw_review_Health_and_Personal_Care',
    'raw_review_Appliances', 'raw_review_Movies_and_TV'
]

# Sampling function
def fetch_sample(config, sample_size=1000):
    try:
        print(f"⏳ Fetching: {config}")
        ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", config, split="full", streaming=True, trust_remote_code=True)
        sample = list(itertools.islice(ds, sample_size))
        df = pd.DataFrame(sample)
        print(f"✅ Done: {config} with {len(df)} rows")
        return df
    except Exception as e:
        print(f"❌ Error loading {config}: {e}")
        return pd.DataFrame()

# Use ThreadPoolExecutor to run in parallel
final_dfs = []
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(fetch_sample, config) for config in configs]
    for future in as_completed(futures):
        result = future.result()
        if not result.empty:
            final_dfs.append(result)

# Combine and limit to 50,000 rows
final_df = pd.concat(final_dfs).sample(frac=1, random_state=42).head(50000).reset_index(drop=True)

print("✅ Final combined shape:", final_df.shape)


⏳ Fetching: raw_review_All_Beauty
⏳ Fetching: raw_review_Toys_and_Games
⏳ Fetching: raw_review_Cell_Phones_and_Accessories
⏳ Fetching: raw_review_Industrial_and_Scientific
✅ Done: raw_review_Toys_and_Games with 1000 rows
⏳ Fetching: raw_review_Gift_Cards
✅ Done: raw_review_Cell_Phones_and_Accessories with 1000 rows
⏳ Fetching: raw_review_Musical_Instruments
✅ Done: raw_review_Industrial_and_Scientific with 1000 rows
⏳ Fetching: raw_review_Electronics
✅ Done: raw_review_All_Beauty with 1000 rows
⏳ Fetching: raw_review_Handmade_Products
✅ Done: raw_review_Gift_Cards with 1000 rows
⏳ Fetching: raw_review_Arts_Crafts_and_Sewing
✅ Done: raw_review_Musical_Instruments with 1000 rows
⏳ Fetching: raw_review_Baby_Products
✅ Done: raw_review_Handmade_Products with 1000 rows
⏳ Fetching: raw_review_Health_and_Household
✅ Done: raw_review_Electronics with 1000 rows
⏳ Fetching: raw_review_Office_Products
✅ Done: raw_review_Arts_Crafts_and_Sewing with 1000 rows
⏳ Fetching: raw_review_Digital_Music
✅ 

In [73]:
final_df.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,1.0,Did not work for us,We have gnats and fruit flys and in one week o...,[],B0B17J715F,B0B17J715F,AE7Y5RLYIKHOZB5NKKOEKYG2SPSQ,1663696345858,2,True
1,5.0,Five Stars,"Great steamer! Wonderful, quality pot.",[],B000058AKE,B0B4FG8MMZ,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1466163389000,0,True
2,5.0,Love this phone case wallet.,I wanted an attractive phone case wallet that ...,[],B09PQSGL7K,B0BX6L3MRY,AEDNNPW6JQ4H6NPMF2FYMIUZOTGQ,1677522366485,0,True
3,2.0,not even close,to good to be true for the price. quality was ...,[],B0048DTDLM,B0048DTDLM,AHSXXZHYMXDRVKC643TJZYQ7XGYA,1333732361000,4,True
4,5.0,kirkland plastic wrap,My local big box has been out of this for too ...,[],B007MWNFBA,B07MTN6R53,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1655841570805,1,True


In [74]:
final_df.describe()

,rating,timestamp,helpful_vote
count,34000.000000,3.400000e+04,34000.000000
mean,4.321029,1.546029e+12,1.871853
std,1.167757,1.061899e+11,12.771998
min,1.000000,9.382045e+11,0.000000
25%,4.000000,1.491181e+12,0.000000
50%,5.000000,1.573600e+12,0.000000
75%,5.000000,1.620423e+12,1.000000
max,5.000000,1.679093e+12,985.000000


In [75]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34000 entries, 0 to 33999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rating             34000 non-null  float64
 1   title              34000 non-null  object 
 2   text               34000 non-null  object 
 3   images             34000 non-null  object 
 4   asin               34000 non-null  object 
 5   parent_asin        34000 non-null  object 
 6   user_id            34000 non-null  object 
 7   timestamp          34000 non-null  int64  
 8   helpful_vote       34000 non-null  int64  
 9   verified_purchase  34000 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 2.4+ MB


In [76]:
final_df['verified_purchase'].value_counts()

,count
verified_purchase,
True,25143
False,8857


In [77]:
final_df.rename(columns={'verified_purchase': 'label'}, inplace=True)
# Step 1: Create a new 'review' column by concatenating 'title' and 'text'
final_df['review'] = final_df['title'] + " " + final_df['text']

# Step 2: Drop the 'title' and 'text' columns
final_df.drop(columns=['title', 'text'], inplace=True)

# Step 3: Verify the changes
print(final_df.head())  # Show the updated DataFrame with the new 'review' column


   rating images        asin parent_asin                       user_id  \
0     1.0     []  B0B17J715F  B0B17J715F  AE7Y5RLYIKHOZB5NKKOEKYG2SPSQ   
1     5.0     []  B000058AKE  B0B4FG8MMZ  AHSPLDNW5OOUK2PLH7GXLACFBZNQ   
2     5.0     []  B09PQSGL7K  B0BX6L3MRY  AEDNNPW6JQ4H6NPMF2FYMIUZOTGQ   
3     2.0     []  B0048DTDLM  B0048DTDLM  AHSXXZHYMXDRVKC643TJZYQ7XGYA   
4     5.0     []  B007MWNFBA  B07MTN6R53  AFZUK3MTBIBEDQOPAK3OATUOUKLA   

       timestamp  helpful_vote  label  \
0  1663696345858             2   True   
1  1466163389000             0   True   
2  1677522366485             0   True   
3  1333732361000             4   True   
4  1655841570805             1   True   

                                              review  
0  Did not work for us We have gnats and fruit fl...  
1  Five Stars Great steamer!  Wonderful, quality ...  
2  Love this phone case wallet. I wanted an attra...  
3  not even close to good to be true for the pric...  
4  kirkland plastic wrap My local

In [78]:
final_df.head()

,rating,images,asin,parent_asin,user_id,timestamp,helpful_vote,label,review
0,1.0,[],B0B17J715F,B0B17J715F,AE7Y5RLYIKHOZB5NKKOEKYG2SPSQ,1663696345858,2,True,Did not work for us We have gnats and fruit fl...
1,5.0,[],B000058AKE,B0B4FG8MMZ,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1466163389000,0,True,"Five Stars Great steamer! Wonderful, quality ..."
2,5.0,[],B09PQSGL7K,B0BX6L3MRY,AEDNNPW6JQ4H6NPMF2FYMIUZOTGQ,1677522366485,0,True,Love this phone case wallet. I wanted an attra...
3,2.0,[],B0048DTDLM,B0048DTDLM,AHSXXZHYMXDRVKC643TJZYQ7XGYA,1333732361000,4,True,not even close to good to be true for the pric...
4,5.0,[],B007MWNFBA,B07MTN6R53,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1655841570805,1,True,kirkland plastic wrap My local big box has bee...


In [79]:
# 1. Calculate mean of valid ratings (1 to 5)
valid_ratings = final_df[(final_df['rating'] >= 1) & (final_df['rating'] <= 5)]
mean_rating = valid_ratings['rating'].mean()

# 2. Replace invalid ratings with mean
final_df['rating'] = final_df['rating'].apply(lambda x: mean_rating if x < 1 or x > 5 else x)


In [80]:
final_df.head()

,rating,images,asin,parent_asin,user_id,timestamp,helpful_vote,label,review
0,1.0,[],B0B17J715F,B0B17J715F,AE7Y5RLYIKHOZB5NKKOEKYG2SPSQ,1663696345858,2,True,Did not work for us We have gnats and fruit fl...
1,5.0,[],B000058AKE,B0B4FG8MMZ,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1466163389000,0,True,"Five Stars Great steamer! Wonderful, quality ..."
2,5.0,[],B09PQSGL7K,B0BX6L3MRY,AEDNNPW6JQ4H6NPMF2FYMIUZOTGQ,1677522366485,0,True,Love this phone case wallet. I wanted an attra...
3,2.0,[],B0048DTDLM,B0048DTDLM,AHSXXZHYMXDRVKC643TJZYQ7XGYA,1333732361000,4,True,not even close to good to be true for the pric...
4,5.0,[],B007MWNFBA,B07MTN6R53,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1655841570805,1,True,kirkland plastic wrap My local big box has bee...


In [81]:
import pandas as pd

# Step 1: Make a deep string-converted version of final_df
temp_df = final_df.copy()

# Convert ALL values in ALL columns to strings (including nested/unhashable types)
temp_df = temp_df.applymap(lambda x: str(x))

# Step 2: Find duplicated rows (including first occurrences)
duplicate_mask = temp_df.duplicated(keep=False)

# Step 3: Extract duplicates from original final_df
duplicates_df = final_df[duplicate_mask]

# Step 4: Display results
print(f"✅ Found {duplicates_df.shape[0]} duplicate rows:")
print(duplicates_df)
# Drop fully duplicated rows from final_df
final_df = final_df[~temp_df.duplicated(keep='first')].reset_index(drop=True)


<ipython-input-81-f09bc4694ad9>:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  temp_df = temp_df.applymap(lambda x: str(x))


✅ Found 6 duplicate rows:
       rating images        asin parent_asin                       user_id  \
2146      5.0     []  B00CZUZ5MG  B00CZUZ5MG  AE5UW4XLVL5AXSJP24MTH4LKK4DA   
11139     1.0     []  B006M3ZVLC  B00QJQUOPK  AGRAB5A32I5FQFZJ5NGTPTSMBXRA   
12731     1.0     []  B006M3ZVLC  B00QJQUOPK  AGRAB5A32I5FQFZJ5NGTPTSMBXRA   
17136     5.0     []  B00BXLSZPM  B00BXLSYEY  AGPIVP6KGLV3RC4MFKELOSHUT76A   
19766     5.0     []  B00BXLSZPM  B00BXLSYEY  AGPIVP6KGLV3RC4MFKELOSHUT76A   
24181     5.0     []  B00CZUZ5MG  B00CZUZ5MG  AE5UW4XLVL5AXSJP24MTH4LKK4DA   

           timestamp  helpful_vote  label  \
2146   1594591814462             3   True   
11139  1621217950771             7   True   
12731  1621217950771             7   True   
17136  1519036358711             1   True   
19766  1519036358711             1   True   
24181  1594591814462             3   True   

                                                  review  
2146                      Easy to use Enjoying the s

In [82]:
# Step 1: Find duplicates in the 'review' column
duplicates_in_review = final_df[final_df['review'].duplicated(keep=False)]

# Step 2: Show how many duplicate reviews we found
print(f"✅ Found {duplicates_in_review.shape[0]} rows with duplicate reviews:")

# Step 3: Display the full rows with duplicated reviews
print(duplicates_in_review)
# Step 4: Remove duplicates and keep only the first occurrence (or you can use keep=False to remove all instances)
final_df = final_df.drop_duplicates(subset=['review'], keep='first')

# Step 5: Verify by printing the DataFrame after removal
print(f"✅ Data after removing duplicates:")
print(final_df)

✅ Found 816 rows with duplicate reviews:
       rating                                             images        asin  \
36        5.0                                                 []  B000G7PNKO   
51        3.0                                                 []  B00004R84V   
56        5.0                                                 []  B086ZWGK7P   
72        4.0  [{'small_image_url': 'https://images-na.ssl-im...  B08JFZVZ8G   
80        5.0                                                 []  B08GM6L9VT   
...       ...                                                ...         ...   
33771     5.0                                                 []  B00BXLVE6Y   
33792     4.0                                                 []  B01N6J0LYA   
33870     1.0                                                 []  B01CBV2K8U   
33948     5.0                                                 []  B07JNF1YNK   
33971     5.0                                                 []  B004Q5T7SQ   

In [83]:
final_df.head()

,rating,images,asin,parent_asin,user_id,timestamp,helpful_vote,label,review
0,1.0,[],B0B17J715F,B0B17J715F,AE7Y5RLYIKHOZB5NKKOEKYG2SPSQ,1663696345858,2,True,Did not work for us We have gnats and fruit fl...
1,5.0,[],B000058AKE,B0B4FG8MMZ,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1466163389000,0,True,"Five Stars Great steamer! Wonderful, quality ..."
2,5.0,[],B09PQSGL7K,B0BX6L3MRY,AEDNNPW6JQ4H6NPMF2FYMIUZOTGQ,1677522366485,0,True,Love this phone case wallet. I wanted an attra...
3,2.0,[],B0048DTDLM,B0048DTDLM,AHSXXZHYMXDRVKC643TJZYQ7XGYA,1333732361000,4,True,not even close to good to be true for the pric...
4,5.0,[],B007MWNFBA,B07MTN6R53,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1655841570805,1,True,kirkland plastic wrap My local big box has bee...


In [84]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 61.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [86]:
import spacy

# Load English model
nlp = spacy.load("en_core_web_sm")

def preprocess_review_spacy(text):
    doc = nlp(text.lower())  # Lowercase and tokenize
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    return ' '.join(tokens)

# Apply to review column
final_df['cleaned_review'] = final_df['review'].apply(preprocess_review_spacy)

# View results
final_df[['review', 'cleaned_review']].head()


,review,cleaned_review
0,Did not work for us We have gnats and fruit fl...,work gnat fruit fly week have sticker butterfl...
1,"Five Stars Great steamer! Wonderful, quality ...",star great steamer wonderful quality pot
2,Love this phone case wallet. I wanted an attra...,love phone case wallet want attractive phone c...
3,not even close to good to be true for the pric...,close good true price quality ok fit close gro...
4,kirkland plastic wrap My local big box has bee...,kirkland plastic wrap local big box long purch...


In [90]:
# Step 1: Create the Product Table with average rating and first image
product_df = final_df.groupby('asin').agg(
    average_rating=('rating', 'mean'),
    images=('images', 'first')  # Assuming all rows of a product have the same image list
).reset_index()



,review_id,rating,asin,user_id,timestamp,label,review,cleaned_review
0,1,1.0,B0B17J715F,AE7Y5RLYIKHOZB5NKKOEKYG2SPSQ,1663696345858,True,Did not work for us We have gnats and fruit fl...,work gnat fruit fly week have sticker butterfl...
1,2,5.0,B000058AKE,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1466163389000,True,"Five Stars Great steamer! Wonderful, quality ...",star great steamer wonderful quality pot
2,3,5.0,B09PQSGL7K,AEDNNPW6JQ4H6NPMF2FYMIUZOTGQ,1677522366485,True,Love this phone case wallet. I wanted an attra...,love phone case wallet want attractive phone c...
3,4,2.0,B0048DTDLM,AHSXXZHYMXDRVKC643TJZYQ7XGYA,1333732361000,True,not even close to good to be true for the pric...,close good true price quality ok fit close gro...
4,5,5.0,B007MWNFBA,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1655841570805,True,kirkland plastic wrap My local big box has bee...,kirkland plastic wrap local big box long purch...


In [94]:
# Step 2: Create the User Table with helpful vote
user_df = final_df[['user_id', 'helpful_vote']].drop_duplicates().reset_index(drop=True)
user_df.head()

,user_id,helpful_vote
0,AE7Y5RLYIKHOZB5NKKOEKYG2SPSQ,2
1,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,0
2,AEDNNPW6JQ4H6NPMF2FYMIUZOTGQ,0
3,AHSXXZHYMXDRVKC643TJZYQ7XGYA,4
4,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1


In [92]:


# Step 3: Create the Review Table and add unique review_id
review_df = final_df.copy()
review_df['review_id'] = review_df.index + 1  # Creating a unique ID based on index

# Assuming cleaned_review was created earlier
# If not, you should run the preprocessing function from before

# Reorder columns to place 'review_id' at the beginning
review_df = review_df[['review_id', 'rating', 'asin', 'user_id', 'timestamp', 'label', 'review', 'cleaned_review']]

# Step 4: Verify the outputs
# print("✅ Product Table:")
product_df.head()review_df.head()

,review_id,rating,asin,user_id,timestamp,label,review,cleaned_review
0,1,1.0,B0B17J715F,AE7Y5RLYIKHOZB5NKKOEKYG2SPSQ,1663696345858,True,Did not work for us We have gnats and fruit fl...,work gnat fruit fly week have sticker butterfl...
1,2,5.0,B000058AKE,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1466163389000,True,"Five Stars Great steamer! Wonderful, quality ...",star great steamer wonderful quality pot
2,3,5.0,B09PQSGL7K,AEDNNPW6JQ4H6NPMF2FYMIUZOTGQ,1677522366485,True,Love this phone case wallet. I wanted an attra...,love phone case wallet want attractive phone c...
3,4,2.0,B0048DTDLM,AHSXXZHYMXDRVKC643TJZYQ7XGYA,1333732361000,True,not even close to good to be true for the pric...,close good true price quality ok fit close gro...
4,5,5.0,B007MWNFBA,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1655841570805,True,kirkland plastic wrap My local big box has bee...,kirkland plastic wrap local big box long purch...
